In [2]:
# 필요한 라이브러리 로드
import pandas as pd
import numpy as np
import pickle

from pylab import rcParams
import matplotlib.pyplot as plt
import seaborn as sns

# import tensorflow as tf
from scipy import stats
# from sklearn.model_selection import train_test_split
# from keras.models import Model, load_model
# from keras.layers import Input, Dense
# from keras.callbacks import ModelCheckpoint, TensorBoard
# from keras import regularizers

# 시각화 라이브러리 설정
%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 14, 8

# RANDOM_SEED와 LABELS 설정
RANDOM_SEED = 42
LABELS = ["Normal", "Fraud"]

> ##### data pre-processing #####

In [1]:
import pandas as pd

filename = '../data/biz_log.csv'
df = pd.read_csv( filename, sep=', ', engine='python', dtype='object')

df.head()

,proc_date,proc_hour,proc_time,emp_cd,emp_nm,user_id,user_ip,dept_cd,dept_nm,org_cd,...,assu_org_cd,assu_org_nm,assu_hq_cd,assu_hq_nm,audit_no,grade,status,log_seq,sys_cd,log_agency_cd
0,20171101,1030,36,20150068,황혜련,20150068,10.1.72.55,DB5,약제기준부,DB0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41596,AB01,HI
1,20171101,1030,35,20150068,황혜련,20150068,10.1.72.55,DB5,약제기준부,DB0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41595,AB01,HI
2,20171101,1026,38,20150068,황혜련,20150068,10.1.72.55,DB5,약제기준부,DB0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41594,AB01,HI
3,20171101,1026,36,20150068,황혜련,20150068,10.1.72.55,DB5,약제기준부,DB0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41593,AB01,HI
4,20171101,1425,21,20162405,박아결,20162405,10.100.4.66,CH9,정보화지원부,CH0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41592,AB01,HI


In [2]:
df.shape

(7786, 82)

In [3]:
df.isna().sum()

proc_date           0
proc_hour           1
proc_time           1
emp_cd              1
emp_nm              1
                 ... 
grade            7786
status           7786
log_seq             1
sys_cd              1
log_agency_cd       1
Length: 82, dtype: int64

In [4]:
# remove columns of all value is null
remove_columns = [key for value, key in zip(df.isna().sum(), df.columns) if value == df.shape[0]]

print( remove_columns )

df.drop(columns=remove_columns, inplace=True)
print(df.isna().sum())


['hq_nm', 'posit_gu', 'acc_org_cd', 'acc_org_nm', 'acc_hq_cd', 'acc_hq_nm', 'bran_cd', 'bran_nm', 'nh_dept_cd', 'nh_dept_nm', 'cd_org_cd', 'cd_org_nm', 'car_cd', 'site_cd', 'member_div', 'ssn_name', 'sear_cont', 'cert_num', 'sear_log24', 'esta_sym', 'firm_nm', 'assu_ssn', 'assu_nm', 'contact_condition', 'psnl_inf_qry_hed_colnm', 'psnl_inf_cnts', 'output_pgm_id', 'inq_db', 'patient_cd', 'patient_nm', 'bloodno', 'result', 'scrn_id', 'scrn_nm', 'vcls', 'ssn_org_cd', 'ssn_org_nm', 'ssn_hq_cd', 'ssn_hq_nm', 'juri_out', 'firm_cd', 'firm_cd_nm', 'firm_hq_cd', 'firm_hq_nm', 'assu_org_cd', 'assu_org_nm', 'assu_hq_cd', 'assu_hq_nm', 'audit_no', 'grade', 'status']
proc_date            0
proc_hour            1
proc_time            1
emp_cd               1
emp_nm               1
user_id              1
user_ip              1
dept_cd              1
dept_nm              1
org_cd               1
org_nm               1
hq_cd                1
join_ssn             1
sear_val          5917
inq_seq         

In [5]:
# remove columns of name
name_columns = list(filter(lambda x: 'nm' in x, df.columns))
# 'prg_nm' ??
print(df[name_columns].isna().sum())
df.drop(columns=name_columns, inplace=True)
print(df.isna().sum())


emp_nm         1
dept_nm        1
org_nm         1
ykiho_nm    3100
busi_nm     7498
prg_nm       289
dtype: int64
proc_date            0
proc_hour            1
proc_time            1
emp_cd               1
user_id              1
user_ip              1
dept_cd              1
org_cd               1
hq_cd                1
join_ssn             1
sear_val          5917
inq_seq              1
per_inf_cd        5917
inq_reason        7341
ykiho_cd          1412
recv_no           5980
recv_yyyy         5980
busi_cd           7498
busi_dtl_contn    7674
button_cd            1
prg_id             289
req_url           7498
log_seq              1
sys_cd               1
log_agency_cd        1
dtype: int64


In [6]:
print( df.info(), len(df.columns), sep='\n' )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7786 entries, 0 to 7785
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   proc_date       7786 non-null   object
 1   proc_hour       7785 non-null   object
 2   proc_time       7785 non-null   object
 3   emp_cd          7785 non-null   object
 4   user_id         7785 non-null   object
 5   user_ip         7785 non-null   object
 6   dept_cd         7785 non-null   object
 7   org_cd          7785 non-null   object
 8   hq_cd           7785 non-null   object
 9   join_ssn        7785 non-null   object
 10  sear_val        1869 non-null   object
 11  inq_seq         7785 non-null   object
 12  per_inf_cd      1869 non-null   object
 13  inq_reason      445 non-null    object
 14  ykiho_cd        6374 non-null   object
 15  recv_no         1806 non-null   object
 16  recv_yyyy       1806 non-null   object
 17  busi_cd         288 non-null    object
 18  busi_dtl

In [7]:
df.head()

,proc_date,proc_hour,proc_time,emp_cd,user_id,user_ip,dept_cd,org_cd,hq_cd,join_ssn,...,recv_no,recv_yyyy,busi_cd,busi_dtl_contn,button_cd,prg_id,req_url,log_seq,sys_cd,log_agency_cd
0,20171101,1030,36,20150068,20150068,10.1.72.55,DB5,DB0,00,dd90baa12f3f6d4bef1a506c856ee5f9,...,NaN,NaN,02,1498,R,NaN,http://biznet.hira.or.kr/ga/gac/selectDramtStd...,41596,AB01,HI
1,20171101,1030,35,20150068,20150068,10.1.72.55,DB5,DB0,00,dd90baa12f3f6d4bef1a506c856ee5f9,...,NaN,NaN,02,NaN,R,NaN,http://biznet.hira.or.kr/ga/gac/selectDramtStd...,41595,AB01,HI
2,20171101,1026,38,20150068,20150068,10.1.72.55,DB5,DB0,00,dd90baa12f3f6d4bef1a506c856ee5f9,...,NaN,NaN,02,NaN,R,NaN,http://biznet.hira.or.kr/ga/gac/selectDramtStd...,41594,AB01,HI
3,20171101,1026,36,20150068,20150068,10.1.72.55,DB5,DB0,00,dd90baa12f3f6d4bef1a506c856ee5f9,...,NaN,NaN,02,NaN,R,NaN,http://biznet.hira.or.kr/ga/gac/selectDramtStd...,41593,AB01,HI
4,20171101,1425,21,20162405,20162405,10.100.4.66,CH9,CH0,00,dd90baa12f3f6d4bef1a506c856ee5f9,...,NaN,NaN,01,invision,R,NaN,http://biznet.hira.or.kr/ga/gae/selectUserInfo.do,41592,AB01,HI


In [8]:
df.hq_cd.unique()

array(['00', '05', '10', '\\N', '01', '06', '07', '09', '03', '04', '02',
       '08', None], dtype=object)

In [9]:
df[df['hq_cd'].isnull()]

,proc_date,proc_hour,proc_time,emp_cd,user_id,user_ip,dept_cd,org_cd,hq_cd,join_ssn,...,recv_no,recv_yyyy,busi_cd,busi_dtl_contn,button_cd,prg_id,req_url,log_seq,sys_cd,log_agency_cd
7785,\.,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [10]:
df.drop(7785, inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7785 entries, 0 to 7784
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   proc_date       7785 non-null   object
 1   proc_hour       7785 non-null   object
 2   proc_time       7785 non-null   object
 3   emp_cd          7785 non-null   object
 4   user_id         7785 non-null   object
 5   user_ip         7785 non-null   object
 6   dept_cd         7785 non-null   object
 7   org_cd          7785 non-null   object
 8   hq_cd           7785 non-null   object
 9   join_ssn        7785 non-null   object
 10  sear_val        1869 non-null   object
 11  inq_seq         7785 non-null   object
 12  per_inf_cd      1869 non-null   object
 13  inq_reason      445 non-null    object
 14  ykiho_cd        6374 non-null   object
 15  recv_no         1806 non-null   object
 16  recv_yyyy       1806 non-null   object
 17  busi_cd         288 non-null    object
 18  busi_dtl

In [12]:
df[df['hq_cd'] == '\\N']

,proc_date,proc_hour,proc_time,emp_cd,user_id,user_ip,dept_cd,org_cd,hq_cd,join_ssn,...,recv_no,recv_yyyy,busi_cd,busi_dtl_contn,button_cd,prg_id,req_url,log_seq,sys_cd,log_agency_cd
550,20171101,1457,49,99000001,99000001,10.1.78.196,\N,\N,\N,dd90baa12f3f6d4bef1a506c856ee5f9,...,9557008,2017,NaN,NaN,R,W_KJC170,NaN,41309,CI01,HI
551,20171101,1434,43,99000001,99000001,10.1.78.196,\N,\N,\N,dd90baa12f3f6d4bef1a506c856ee5f9,...,8491593,2017,NaN,NaN,R,W_KJC170,NaN,41308,CI01,HI


In [13]:
df.drop([550, 551], inplace=True)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7783 entries, 0 to 7784
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   proc_date       7783 non-null   object
 1   proc_hour       7783 non-null   object
 2   proc_time       7783 non-null   object
 3   emp_cd          7783 non-null   object
 4   user_id         7783 non-null   object
 5   user_ip         7783 non-null   object
 6   dept_cd         7783 non-null   object
 7   org_cd          7783 non-null   object
 8   hq_cd           7783 non-null   object
 9   join_ssn        7783 non-null   object
 10  sear_val        1867 non-null   object
 11  inq_seq         7783 non-null   object
 12  per_inf_cd      1867 non-null   object
 13  inq_reason      445 non-null    object
 14  ykiho_cd        6372 non-null   object
 15  recv_no         1804 non-null   object
 16  recv_yyyy       1804 non-null   object
 17  busi_cd         288 non-null    object
 18  busi_dtl

In [15]:
df['sear_val'].unique()

array([nan, '의료기관=36100641,접수번호=9451242,접수년도=2017',
       '의료기관=36202762,접수번호=9306402,접수년도=2017', ...,
       '의료기관=31286348,접수번호=9526949,접수년도=2017',
       '의료기관=31100058,접수번호=9284053,접수년도=2017',
       '의료기관=31205861,접수번호=9359977,접수년도=2017,지원코드=00,청일련=1,명일련=0000500'],
      dtype=object)

In [16]:
df['sear_val'].fillna('', inplace=True )

In [17]:


print(dict(df.isna().sum()))

{'proc_date': 0, 'proc_hour': 0, 'proc_time': 0, 'emp_cd': 0, 'user_id': 0, 'user_ip': 0, 'dept_cd': 0, 'org_cd': 0, 'hq_cd': 0, 'join_ssn': 0, 'sear_val': 0, 'inq_seq': 0, 'per_inf_cd': 5916, 'inq_reason': 7338, 'ykiho_cd': 1411, 'recv_no': 5979, 'recv_yyyy': 5979, 'busi_cd': 7495, 'busi_dtl_contn': 7671, 'button_cd': 0, 'prg_id': 288, 'req_url': 7495, 'log_seq': 0, 'sys_cd': 0, 'log_agency_cd': 0}


In [18]:
any_null_column = []
for key, val in dict(df.isna().sum()).items():
    if bool(val):
        any_null_column.append(key)

print( any_null_column )

['per_inf_cd', 'inq_reason', 'ykiho_cd', 'recv_no', 'recv_yyyy', 'busi_cd', 'busi_dtl_contn', 'prg_id', 'req_url']


In [19]:
df[any_null_column ].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7783 entries, 0 to 7784
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   per_inf_cd      1867 non-null   object
 1   inq_reason      445 non-null    object
 2   ykiho_cd        6372 non-null   object
 3   recv_no         1804 non-null   object
 4   recv_yyyy       1804 non-null   object
 5   busi_cd         288 non-null    object
 6   busi_dtl_contn  112 non-null    object
 7   prg_id          7495 non-null   object
 8   req_url         288 non-null    object
dtypes: object(9)
memory usage: 608.0+ KB


In [20]:
df.busi_cd.fillna('', inplace=True)

In [21]:
df.busi_cd

0       02
1       02
2       02
3       02
4       01
        ..
7780      
7781      
7782      
7783      
7784    01
Name: busi_cd, Length: 7783, dtype: object

In [22]:
for key in any_null_column:
    print( '='*10 )
    print( f'{key} : ', end='')
    print( f'{df[key].dtypes} : ', end='')
    print(  df[key].unique() )

per_inf_cd : object : [nan 'FZ' 'FZ1317' 'FZ1311']
inq_reason : object : [nan '(심사)명세서심사 관련 확인' '동일 환자 연계심사 주민번호 확인' '건강보험에 직장가입한 여부 확인'
 '심사시 자격변동(DC조회) 및 사망여부 확인' '(심사)사후관리 업무관련 확인' '동일 수진자 연계심사 주민번호 확인'
 '응급대불청구서접수를위한수진자자격조회' '민원관련업무' '공단 자격 확인(심사불능 등)' '진료비민원처리'
 '의료급여 정신과 정액수가 사후관리 관련 확인' '건강보험등 자격변동(DC조회) 및 사망여부 조회' '이중자격자 확인'
 '현지조사 분석' '기타 현지조사업무' '(심사)진료비확인 등 민원관련 확인' '진료비민원 접수를 위한 확인']
ykiho_cd : object : [nan '36100641' '36202762' ... '37282379' '33811636' '34343288']
recv_no : object : [nan '9451242' '9306402' ... '9526949' '9284053' '9359977']
recv_yyyy : object : [nan '2017' '2016' '2015' '2013' '2014']
busi_cd : object : ['02' '01' '' '26' '10']
busi_dtl_contn : object : ['1498' nan 'invision' 'mch198383' '1512' '1503' 'olly0639' '524' '1496'
 '1430' '2546' '1515' 'ykyk2833' '1499' '1511' '1519' '1518' '1513'
 'mhy1430' 'pedchask' '1509' '1514' 'jwobgy1' 'jongwonn96' '52015' '1389'
 '1446' 'seoulrise1' '2545' 'ks0621' 'nurim2529' '1296' '1516' '1517'
 '1520' '525' '1309'

In [23]:
for key in any_null_column:
    if df[key].dtypes == 'object':
        df[key].fillna('', inplace=True )        

In [24]:
for key in any_null_column:
    print( '='*10 )
    print( f'{key} : ', end='')
    print( f'{df[key].dtypes} : ', end='')
    print(  df[key].unique() )

per_inf_cd : object : ['' 'FZ' 'FZ1317' 'FZ1311']
inq_reason : object : ['' '(심사)명세서심사 관련 확인' '동일 환자 연계심사 주민번호 확인' '건강보험에 직장가입한 여부 확인'
 '심사시 자격변동(DC조회) 및 사망여부 확인' '(심사)사후관리 업무관련 확인' '동일 수진자 연계심사 주민번호 확인'
 '응급대불청구서접수를위한수진자자격조회' '민원관련업무' '공단 자격 확인(심사불능 등)' '진료비민원처리'
 '의료급여 정신과 정액수가 사후관리 관련 확인' '건강보험등 자격변동(DC조회) 및 사망여부 조회' '이중자격자 확인'
 '현지조사 분석' '기타 현지조사업무' '(심사)진료비확인 등 민원관련 확인' '진료비민원 접수를 위한 확인']
ykiho_cd : object : ['' '36100641' '36202762' ... '37282379' '33811636' '34343288']
recv_no : object : ['' '9451242' '9306402' ... '9526949' '9284053' '9359977']
recv_yyyy : object : ['' '2017' '2016' '2015' '2013' '2014']
busi_cd : object : ['02' '01' '' '26' '10']
busi_dtl_contn : object : ['1498' '' 'invision' 'mch198383' '1512' '1503' 'olly0639' '524' '1496'
 '1430' '2546' '1515' 'ykyk2833' '1499' '1511' '1519' '1518' '1513'
 'mhy1430' 'pedchask' '1509' '1514' 'jwobgy1' 'jongwonn96' '52015' '1389'
 '1446' 'seoulrise1' '2545' 'ks0621' 'nurim2529' '1296' '1516' '1517'
 '1520' '525' '1309' '1095

In [25]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7783 entries, 0 to 7784
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   proc_date       7783 non-null   object
 1   proc_hour       7783 non-null   object
 2   proc_time       7783 non-null   object
 3   emp_cd          7783 non-null   object
 4   user_id         7783 non-null   object
 5   user_ip         7783 non-null   object
 6   dept_cd         7783 non-null   object
 7   org_cd          7783 non-null   object
 8   hq_cd           7783 non-null   object
 9   join_ssn        7783 non-null   object
 10  sear_val        7783 non-null   object
 11  inq_seq         7783 non-null   object
 12  per_inf_cd      7783 non-null   object
 13  inq_reason      7783 non-null   object
 14  ykiho_cd        7783 non-null   object
 15  recv_no         7783 non-null   object
 16  recv_yyyy       7783 non-null   object
 17  busi_cd         7783 non-null   object
 18  busi_dtl

In [26]:
df['proc_datetime'] =  df['proc_date'] + ' ' + df[ 'proc_hour'] + df['proc_time']

In [54]:
df.proc_date.unique()

array(['20171101', '20171113'], dtype=object)

In [27]:
df[['proc_date', 'proc_hour', 'proc_time', 'proc_datetime']]

,proc_date,proc_hour,proc_time,proc_datetime
0,20171101,1030,36,20171101 103036
1,20171101,1030,35,20171101 103035
2,20171101,1026,38,20171101 102638
3,20171101,1026,36,20171101 102636
4,20171101,1425,21,20171101 142521
...,...,...,...,...
7780,20171101,0006,50,20171101 000650
7781,20171101,0004,32,20171101 000432
7782,20171101,0001,37,20171101 000137
7783,20171101,0000,35,20171101 000035


In [28]:
df.proc_datetime =  pd.to_datetime(df['proc_datetime'], format='%Y%m%d %H%M%S')
df.proc_datetime

0      2017-11-01 10:30:36
1      2017-11-01 10:30:35
2      2017-11-01 10:26:38
3      2017-11-01 10:26:36
4      2017-11-01 14:25:21
               ...        
7780   2017-11-01 00:06:50
7781   2017-11-01 00:04:32
7782   2017-11-01 00:01:37
7783   2017-11-01 00:00:35
7784   2017-11-13 11:36:32
Name: proc_datetime, Length: 7783, dtype: datetime64[ns]

In [29]:
df[['proc_date', 'proc_hour', 'proc_time', 'proc_datetime']]

,proc_date,proc_hour,proc_time,proc_datetime
0,20171101,1030,36,2017-11-01 10:30:36
1,20171101,1030,35,2017-11-01 10:30:35
2,20171101,1026,38,2017-11-01 10:26:38
3,20171101,1026,36,2017-11-01 10:26:36
4,20171101,1425,21,2017-11-01 14:25:21
...,...,...,...,...
7780,20171101,0006,50,2017-11-01 00:06:50
7781,20171101,0004,32,2017-11-01 00:04:32
7782,20171101,0001,37,2017-11-01 00:01:37
7783,20171101,0000,35,2017-11-01 00:00:35


In [30]:
len(df['join_ssn'].unique())

76

In [31]:
df.log_agency_cd.unique()

array(['HI'], dtype=object)

In [32]:
df.drop( columns=[ 'log_agency_cd', ], inplace=True)

In [33]:
df.per_inf_cd.unique()

array(['', 'FZ', 'FZ1317', 'FZ1311'], dtype=object)

In [34]:
df.head(3)

,proc_date,proc_hour,proc_time,emp_cd,user_id,user_ip,dept_cd,org_cd,hq_cd,join_ssn,...,recv_no,recv_yyyy,busi_cd,busi_dtl_contn,button_cd,prg_id,req_url,log_seq,sys_cd,proc_datetime
0,20171101,1030,36,20150068,20150068,10.1.72.55,DB5,DB0,00,dd90baa12f3f6d4bef1a506c856ee5f9,...,,,02,1498,R,,http://biznet.hira.or.kr/ga/gac/selectDramtStd...,41596,AB01,2017-11-01 10:30:36
1,20171101,1030,35,20150068,20150068,10.1.72.55,DB5,DB0,00,dd90baa12f3f6d4bef1a506c856ee5f9,...,,,02,,R,,http://biznet.hira.or.kr/ga/gac/selectDramtStd...,41595,AB01,2017-11-01 10:30:35
2,20171101,1026,38,20150068,20150068,10.1.72.55,DB5,DB0,00,dd90baa12f3f6d4bef1a506c856ee5f9,...,,,02,,R,,http://biznet.hira.or.kr/ga/gac/selectDramtStd...,41594,AB01,2017-11-01 10:26:38


In [35]:
data = df[df.columns[3:]]
data

,emp_cd,user_id,user_ip,dept_cd,org_cd,hq_cd,join_ssn,sear_val,inq_seq,per_inf_cd,...,recv_no,recv_yyyy,busi_cd,busi_dtl_contn,button_cd,prg_id,req_url,log_seq,sys_cd,proc_datetime
0,20150068,20150068,10.1.72.55,DB5,DB0,00,dd90baa12f3f6d4bef1a506c856ee5f9,,6626256,,...,,,02,1498,R,,http://biznet.hira.or.kr/ga/gac/selectDramtStd...,41596,AB01,2017-11-01 10:30:36
1,20150068,20150068,10.1.72.55,DB5,DB0,00,dd90baa12f3f6d4bef1a506c856ee5f9,,6626255,,...,,,02,,R,,http://biznet.hira.or.kr/ga/gac/selectDramtStd...,41595,AB01,2017-11-01 10:30:35
2,20150068,20150068,10.1.72.55,DB5,DB0,00,dd90baa12f3f6d4bef1a506c856ee5f9,,6626254,,...,,,02,,R,,http://biznet.hira.or.kr/ga/gac/selectDramtStd...,41594,AB01,2017-11-01 10:26:38
3,20150068,20150068,10.1.72.55,DB5,DB0,00,dd90baa12f3f6d4bef1a506c856ee5f9,,6626253,,...,,,02,,R,,http://biznet.hira.or.kr/ga/gac/selectDramtStd...,41593,AB01,2017-11-01 10:26:36
4,20162405,20162405,10.100.4.66,CH9,CH0,00,dd90baa12f3f6d4bef1a506c856ee5f9,,6626379,,...,,,01,invision,R,,http://biznet.hira.or.kr/ga/gae/selectUserInfo.do,41592,AB01,2017-11-01 14:25:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7780,20060107,20060107,10.100.20.176,MB2,MB0,00,dd90baa12f3f6d4bef1a506c856ee5f9,,27124648,,...,,,,,R,JC0750,,29765,INTR,2017-11-01 00:06:50
7781,19930011,19930011,10.6.1.104,R34,R30,05,dd90baa12f3f6d4bef1a506c856ee5f9,,27124647,,...,,,,,R,JC0152,,29764,INTR,2017-11-01 00:04:32
7782,19930011,19930011,10.6.1.104,R34,R30,05,dd90baa12f3f6d4bef1a506c856ee5f9,,27124646,,...,,,,,R,JC0152,,29763,INTR,2017-11-01 00:01:37
7783,19930011,19930011,10.6.1.104,R34,R30,05,dd90baa12f3f6d4bef1a506c856ee5f9,,27124645,,...,,,,,R,JC0152,,29762,INTR,2017-11-01 00:00:35


In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7783 entries, 0 to 7784
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   emp_cd          7783 non-null   object        
 1   user_id         7783 non-null   object        
 2   user_ip         7783 non-null   object        
 3   dept_cd         7783 non-null   object        
 4   org_cd          7783 non-null   object        
 5   hq_cd           7783 non-null   object        
 6   join_ssn        7783 non-null   object        
 7   sear_val        7783 non-null   object        
 8   inq_seq         7783 non-null   object        
 9   per_inf_cd      7783 non-null   object        
 10  inq_reason      7783 non-null   object        
 11  ykiho_cd        7783 non-null   object        
 12  recv_no         7783 non-null   object        
 13  recv_yyyy       7783 non-null   object        
 14  busi_cd         7783 non-null   object        
 15  busi

In [37]:
data.loc[(data['recv_yyyy'].str.len() > 0)]['recv_yyyy'].unique()

array(['2017', '2016', '2015', '2013', '2014'], dtype=object)

> ##### save dataframe to csv, json

In [38]:

data.to_json('../data/biz_log.ndjson', orient='records', force_ascii=False, lines=True)
data.to_csv('../data/biz_log.json.csv', sep=',', index=False)

> ##### load dataframe to csv, json

In [39]:
# filename = '../data/biz_log.json.csv'
# data = pd.read_csv( filename, sep=',', engine='python', dtype='object', )

> ##### opensearch declare `client` ######

In [40]:
%load_ext autoreload
%autoreload 2

In [41]:
# import this to stop opensearch-py-ml from yelling every time a DataFrame connection made
import warnings
warnings.filterwarnings('ignore')

In [42]:
# imports to demonstrate DataFrame support
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import opensearch_py_ml as oml
from opensearchpy import OpenSearch

# Import standard test settings for consistent results
from opensearch_py_ml.conftest import *

In [43]:
# CLUSTER_URL = 'https://localhost:9200'
CLUSTER_URL = 'default-opensearch-node1-0c190-1595032-15144b153298.kr-gov.lb.naverncp.com'

def get_os_client(cluster_url = CLUSTER_URL,
                  username='admin',
                  password='admin'):
    '''
    Get OpenSearch client
    :param cluster_url: cluster URL like https://ml-te-netwo-1s12ba42br23v-ff1736fa7db98ff2.elb.us-west-2.amazonaws.com:443
    :return: OpenSearch client
    '''
    client = OpenSearch(
        hosts = [{"host": "default-opensearch-node1-0c190-1595032-15144b153298.kr-gov.lb.naverncp.com", "port": 9200}],
        http_auth = ("admin", "admin"),
        use_ssl = True,
        verify_certs = False,
        ssl_assert_hostname = False,
        ssl_show_warn = False,
    )
    
    return client

In [44]:
client = get_os_client()

In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7783 entries, 0 to 7784
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   emp_cd          7783 non-null   object        
 1   user_id         7783 non-null   object        
 2   user_ip         7783 non-null   object        
 3   dept_cd         7783 non-null   object        
 4   org_cd          7783 non-null   object        
 5   hq_cd           7783 non-null   object        
 6   join_ssn        7783 non-null   object        
 7   sear_val        7783 non-null   object        
 8   inq_seq         7783 non-null   object        
 9   per_inf_cd      7783 non-null   object        
 10  inq_reason      7783 non-null   object        
 11  ykiho_cd        7783 non-null   object        
 12  recv_no         7783 non-null   object        
 13  recv_yyyy       7783 non-null   object        
 14  busi_cd         7783 non-null   object        
 15  busi

> ##### Create opensearch biz_log_v2 index #####

In [46]:
index_name = 'biz_log_v5'

In [47]:
import json 

properties = dict()
for column in data.columns:
    text = 'ip' if 'user_ip' in column else \
           'date' if 'proc_datetime' in column else 'text'
    
    properties.update( { column : { 'type' : text } } )

properties

body = {
  "mappings": {
    "properties": properties
  }
}

print(json.dumps( body, indent=2))
response = client.indices.create(index_name, body=body)

{
  "mappings": {
    "properties": {
      "emp_cd": {
        "type": "text"
      },
      "user_id": {
        "type": "text"
      },
      "user_ip": {
        "type": "ip"
      },
      "dept_cd": {
        "type": "text"
      },
      "org_cd": {
        "type": "text"
      },
      "hq_cd": {
        "type": "text"
      },
      "join_ssn": {
        "type": "text"
      },
      "sear_val": {
        "type": "text"
      },
      "inq_seq": {
        "type": "text"
      },
      "per_inf_cd": {
        "type": "text"
      },
      "inq_reason": {
        "type": "text"
      },
      "ykiho_cd": {
        "type": "text"
      },
      "recv_no": {
        "type": "text"
      },
      "recv_yyyy": {
        "type": "text"
      },
      "busi_cd": {
        "type": "text"
      },
      "busi_dtl_contn": {
        "type": "text"
      },
      "button_cd": {
        "type": "text"
      },
      "prg_id": {
        "type": "text"
      },
      "req_url": {
        "typ

> ##### Create opensearch biz_log_v2 body #####

In [48]:
bulk_data = []

for i, row in data.iterrows():
    body = dict()
    for key, value in row.items():
        body.update( { key : value } )

        bulk_data.append({
            "_index": index_name,
            "_id": i,
            "_source": body
        })

     
    # print( body )

> ##### upload to opensearch by bulk #####

In [49]:
from opensearchpy.helpers import bulk

bulk(client, bulk_data)

(171226, [])

In [50]:
# client.indices.refresh( index='biz_logs_v2' )
client.cat.count( index=index_name, format='json')

[{'epoch': '1681375819', 'timestamp': '08:50:19', 'count': '7319'}]

> ##### Upload to opensearch by opensearch_ml.dataframe #####
- using opensearch_py_ml.pandas_to_opensearch

In [51]:
# df2oml = oml.pandas_to_opensearch( df,

#                         #    OPENSEARCH_TEST_CLIENT,
#                            client,

#                            'pandas_to_opensearch',

#                            os_if_exists="replace",

#                            os_refresh=True) # index field 'H' as text not keyword

In [52]:
# df2oml.info()

> ##### delete index opensearch server #####

In [53]:
# client.indices.delete( index=index_name )